In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
NAO_df = pd.read_csv(
    r"C:\Users\Niebaum\Documents\Repositories\reconstruct-climate-indices\data\observations\NAO\nao.csv",
    delim_whitespace=True,
    skiprows=1,
    header=None,
    index_col=0,
    skipfooter=3,
    # encoding='utf-16'
)
NAO_df
NAO_df.index = pd.to_datetime(NAO_df.index)
NAO_df.index.name = "Year"


NAO_np = NAO_df.to_numpy().flatten()
NAO_date = pd.date_range(start="1948-01-01", periods=len(NAO_np), freq="MS")

C:\Users\Niebaum\AppData\Local\Temp\ipykernel_9632\3217758463.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  NAO_df = pd.read_csv(


In [ ]:
AMO_df = pd.read_csv(
    r"C:\Users\Niebaum\Documents\Repositories\reconstruct-climate-indices\data\observations\AMO\amo.csv",
    delim_whitespace=True,
    skiprows=1,
    header=None,
    index_col=0,
    skipfooter=4,
    # encoding='utf-16'
)
AMO_df
AMO_df.index = pd.to_datetime(AMO_df.index)
AMO_df.index.name = "Year"


AMO_np = AMO_df.to_numpy().flatten()
# AMO_np = np.tile(AMO_np, 3)
# NAO_np = np.tile(NAO_np, 3)

AMO_date = pd.date_range(start="1948-01-01", periods=len(AMO_np), freq="MS")

C:\Users\Niebaum\AppData\Local\Temp\ipykernel_9632\1580544381.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  AMO_df = pd.read_csv(


In [ ]:
data = xr.Dataset(
    data_vars=dict(NAO=(["time"], NAO_np), AMO=(["time"], AMO_np)),
    coords=dict(time=(["time"], AMO_date)),
)
for var in data.data_vars:
    data[var][data[var] == -99.99] = np.nan
    data[var][data[var] == -99.9] = np.nan


mask = np.isfinite(data)
finite_mask = np.logical_and(mask.AMO.values, mask.NAO.values)
data = data.sel(time=data.time[finite_mask])


from kalman_reconstruction.pipeline import (
    add_random_variable,
    xarray_Kalman_SEM,
    from_standard_dataset,
)
from kalman_reconstruction.statistics import normalize

add_random_variable(
    ds=data,
    var_name="latent",
    random_generator=np.random.default_rng(seed=10000),
    dim="time",
    variance=1,
)
data["sin"] = (
    np.sin(2 * np.pi * np.arange(0, len(data.time)) / (10 * 12)) + data.AMO * 0
)
data["sin"].plot()
res = xarray_Kalman_SEM(
    ds=data,
    observation_variables=["AMO", "NAO"],
    state_variables=["AMO", "NAO", "latent"],
    nb_iter_SEM=30,
)

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:11<00:00,  2.57it/s]


In [ ]:
res["log_likelihod"].plot()
res_kalman_states = from_standard_dataset(res)

In [ ]:
import matplotlib.pyplot as plt

for var in ["NAO", "AMO"]:
    plt.scatter(data[var], res_kalman_states["latent"], label=var)

In [ ]:
import numpy as np
from pathlib import Path
from kalman_reconstruction.custom_plot import (
    set_custom_rcParams,
    adjust_lightness,
    handler_map_alpha,
    plot_colors,
    symmetrize_axis,
)
from reconstruct_climate_indices.idealized_ocean import spunge_ocean, oscillatory_ocean
from reconstruct_climate_indices.statistics import linear_regression_loglog
import matplotlib.pyplot as plt
from kalman_reconstruction.statistics import normalize

# from sklearn.linear_model import LinearRegression
from scipy import signal

In [ ]:
150 * 365.25 / 30.4375

1800.0

In [ ]:
# plt.style.use('default')
# dark_color = [0.2, 0.2, 0.2]
# light_color = [0.9, 0.9, 0.9]
# lightness_0 = 0.75
# lightness_1 = 0.5
plt.style.use("dark_background")
dark_color = [0.7, 0.7, 0.7]
light_color = [0.1, 0.1, 0.1]
lightness_0 = 1.15
lightness_1 = 1.5
set_custom_rcParams()
plt.rcParams["axes.grid"] = False
colors = [
    "#CC6677",
    "#6E9CB3",
    "#CA8727",
    "#44AA99",
    "#AA4499",
    "#D6BE49",
    "#A494F5",
]
plot_colors(colors)
variables_color = dict()
variables_color["NAO"] = colors[0]
variables_color["AMO"] = colors[2]
variables_color["sin"] = colors[1]
variables_color["latent"] = colors[3]

In [ ]:
fs = 12
welch_window_width = 100
fig, axs = plt.subplots(
    nrows=2, ncols=1, layout="constrained", sharex=True, sharey=True
)
kwargs = dict(
    linestyle="-",
    linewidth="1",
    marker=".",
)

axs = axs.flatten()
idx = 0  # x index to plot the text at

axs[0].set_title("Observaions")

# ------------------
# plot SAT
# ------------------
for var in ["NAO", "AMO", "latent"]:
    color = variables_color[var]
    x = data[var].values
    frequencies, spectrum = signal.welch(x=x, fs=fs, window="hann", nperseg=len(x))
    axs[0].loglog(frequencies, spectrum, label=f"{var}", color=color, alpha=0.7)
    # frequencies, spectrum = signal.welch(
    #         x = x,
    #         fs=fs,
    #         window="hann",
    #         nperseg = int(welch_window_width*fs)
    #     )
    # axs[0].loglog(
    #     frequencies,
    #     spectrum,
    #     label = f"{var} welch",
    #     color = adjust_lightness(color, lightness_0),
    # )
    frequencies_linear, spectrum_linear, regression = linear_regression_loglog(
        frequencies=frequencies, spectrum=spectrum, weights="f_inverse"
    )
    slope = regression.coef_[0, 0]
    axs[0].loglog(
        frequencies_linear,
        spectrum_linear,
        color=adjust_lightness(color, lightness_1),
    )
    axs[0].text(
        frequencies_linear[idx],
        spectrum_linear[idx],
        f"m={slope:.2f}",
        ha="right",
        va="bottom",
        bbox=dict(facecolor=light_color, edgecolor="None", alpha=0.25),
        color=adjust_lightness(color, lightness_1),
    )


for var in ["NAO", "AMO", "latent"]:
    color = variables_color[var]
    x = res_kalman_states[var].values
    frequencies, spectrum = signal.welch(x=x, fs=fs, window="hann", nperseg=len(x))
    axs[1].loglog(frequencies, spectrum, label=f"{var}", color=color, alpha=0.7)
    # frequencies, spectrum = signal.welch(
    #         x = x,
    #         fs=fs,
    #         window="hann",
    #         nperseg = int(welch_window_width*fs)
    #     )
    # axs[0].loglog(
    #     frequencies,
    #     spectrum,
    #     label = f"{var} welch",
    #     color = adjust_lightness(color, lightness_0),
    # )
    frequencies_linear, spectrum_linear, regression = linear_regression_loglog(
        frequencies=frequencies, spectrum=spectrum, weights="f_inverse"
    )
    slope = regression.coef_[0, 0]
    axs[1].loglog(
        frequencies_linear,
        spectrum_linear,
        color=adjust_lightness(color, lightness_1),
    )
    axs[1].text(
        frequencies_linear[idx],
        spectrum_linear[idx],
        f"m={slope:.2f}",
        ha="right",
        va="bottom",
        bbox=dict(facecolor=light_color, edgecolor="None", alpha=0.25),
        color=adjust_lightness(color, lightness_1),
    )


for ax in axs.flatten():
    ax.axvline(
        1 / welch_window_width, color=dark_color, linestyle="-"
    )  # , label = f"welch window.:\n{welch_window_width}")
    ax.text(
        1 / welch_window_width,
        10 ** (-6.5),
        r"$f_{welch}$",
        ha="right",
        va="bottom",
        c=dark_color,
        rotation=90,
    )
    ax.legend(ncol=2, labelspacing=0.01, loc="lower left")

# fig.suptitle(f"$\tau_0$:{tau0} years | $\omega_0$:{omega0} years | welch_windon:{welch_window_width} years | duration {years} years")

# save_fig(fig=fig, relative_path=f"Frequency_{years}y.pdf")